# Recommendation System on Movie

Library Import

In [40]:
import numpy as np
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

## Data Loading

Dataset Preparation

Note that dataset can be found here: https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [2]:
dataset_link = "tmdb_5000_movies.csv"

df = pd.read_csv(dataset_link)
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

Define Each Fields

In [3]:
id_field = "id"
budget_field = "budget"
genres_field = "genres"
homepage_field = "homepage"
keyword_field = "keywords"
original_language_field = "original_language"
original_title_field = "original_title"
overview_field = "overview"
popularity_field = "popularity"
production_companies_field = "production_companies"
production_countries_field = "production_countries"
release_date_field = "release_date"
revenue_field = "revenue"
runtime_field = "runtime"
spoken_languages_field = "spoken_languages"
status_field = "status"
tagline_field = "tagline"
title_field = "title"
vote_average_field = "vote_average"
vote_count_field = "vote_count"

Define Fields

Note that we also remove unecessary fields.

In [48]:
fields = df.columns.to_list()
target = original_title_field

fields.remove(id_field)
fields.remove(homepage_field)
fields.remove(title_field)
fields.remove(original_title_field)
fields.remove(revenue_field)
fields.remove(vote_count_field)

print(fields)

['budget', 'genres', 'keywords', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count']


View Statistics for Each Fields

In [49]:
df[fields].describe()

,budget,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


## Data Preprocessing

### Convert Number Fields to Word

### Convert JSON in fields of Movie Dataset into string

In [5]:
def convert_json_field_in_df_into_strings(df, df_field, json_field):

    """
    Convert JSON fields to string, by taking the dataframe and a field, a json field, which returns a list of string

    from a specific field in a dataframe

    Below are the explanation of parameters used.
    * df: dataframe to be used
    * df_field: field in dataframe, to convert json into strings
    * json_field: one field to be kept from json inside the data field in dataframe.

    Consider the example:

    | id | movie_name | movie_genres |

    | 1 | a           | {"id": 1, "name": "action"}, {"id": 2, "name": "romance"} | 

    | 2 | b           | {"id": 1, "name": "slice of life"}, {"id": 2, "name": "food"} |

    If we use the function: 
    
    convert_json_field_in_df_into_strings(df, "movie_genres", "name")

    This gives us the list of string from all rows in movie_genres field.

    """

    json_data = json.loads(df[df_field].to_json())
    json_values = list(json_data.values())

    list_of_strings = []

    for value in json_values:

        my_string = ""

        dicts = list(json.loads(value))[:]

        for dict_value in dicts:
            my_string += f"{dict_value[json_field]} "
        list_of_strings.append(my_string[:-1])

    return list_of_strings

copy_df = df.copy()

json_field = "name"

copy_df[genres_field] = convert_json_field_in_df_into_strings(df, genres_field, json_field)
copy_df[keyword_field] = convert_json_field_in_df_into_strings(df, keyword_field, json_field)
copy_df[spoken_languages_field] = convert_json_field_in_df_into_strings(df, spoken_languages_field, json_field)
copy_df[production_companies_field] = convert_json_field_in_df_into_strings(df, production_companies_field, json_field)
copy_df[production_countries_field] = convert_json_field_in_df_into_strings(df, production_countries_field, json_field)

copy_df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners Twentieth Century Fox ...,United States of America United Kingdom,2009-12-10,2787965087,162.0,English Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures Jerry Bruckheimer Films S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures Danjaq B24,United Kingdom United States of America,2015-10-26,880674609,148.0,Français English Español Italiano Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures Warner Bros. DC Entertainme...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,Action Crime Thriller,NaN,9367,united states–mexico barrier legs arms paper k...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,Columbia Pictures,Mexico United States of America,1992-09-04,2040920,81.0,Español,Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,Comedy Romance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,,,2011-12-26,0,85.0,,Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,Front Street Pictures Muse Entertainment Enter...,United States of America,2013-10-13,0,120.0,English,Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,,United States of America China,2012-05-03,0,98.0,English,Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7


### Combine All Strings into One Text

In [31]:
def combine_all_strings_from_df(df, fields, whitespace_split = ','):
    """
    Combine all strings from one df and all fields. 
    
    If you'd like to put something between words, provide whitespace_split parameter.
    """
    df_strings = df.to_string(header = False,
                       index = False,
                       columns = fields,
                       index_names = False).split('\n')

    all_strings = [" ".join(s.split()) for s in df_strings]

    return all_strings

df_strings = combine_all_strings_from_df(copy_df, fields)

print(df_strings[0])

237000000 Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d en Avatar In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. 150.437577 Ingenious Film Partners Twentieth Century Fox Film Corporation Dune Entertainment Lightstorm Entertainment United States of America United Kingdom 2009-12-10 2787965087 162.0 English Español Released Enter the World of Pandora. Avatar 7.2 11800


### Transform the Text into TfIdfVectorizer

In [47]:
tf_idf_vectorizer = TfidfVectorizer()
vector_components = tf_idf_vectorizer.fit_transform(df_strings)

print("Vector Components Size:", vector_components.shape)
print("Number of Non-Empty Elements:", (vector_components != 0).sum())

Vector Components Size: (4803, 37405)
Number of Non-Empty Elements: 379191


### Predict Model